# Simulating the power

### Importing the necessasy packages

In [ ]:
import pvlib
import pandas as pd

### Importing measured temperature from the reference cells

In [ ]:
temp = pd.read_csv(r'path...\filename.csv', sep = ';')                   # Importing the file with measured temperature
temp = temp.set_index('Timestamp')                                       # Setting the timestamps to be the index
temp.index = pd.to_datetime(temp.index)                                  # Converting the index to date-time

# Localizing the timezone and handling daylight saving time
temp = temp.tz_localize('CET', ambiguous = False, nonexistent = 'NaT')  
temp.loc[temp.index.dropna()]                                            # Removing NaN values
temp = temp.tz_convert('UTC')                                            # Converting the timezone to UTC

temp = temp.loc[temp.index.strftime('%M:%S') == '30:00']                 # Selecting hourly data for 11:30 PM, 12:30 AM etc.

temp['average'] = temp.mean(axis = 1)                                    # Calculating the average temperature

For simulating the power, it was assumed that all the solar panels have the same, average temperature.

### Importing measured irradiance from the reference cells

In [ ]:
irr_ref = pd.read_csv(r'path...\filename.csv', sep = ';')    
irr_ref = irr_ref.set_index('Timestamp')
irr_ref.index = pd.to_datetime(irr_ref.index)

irr_ref = irr_ref.tz_localize('CET', ambiguous = False, nonexistent = 'NaT')
irr_ref.loc[irr_ref.index.dropna()]
irr_ref = irr_ref.tz_convert('UTC')

irr_ref = irr_ref.loc[irr_ref.index.strftime('%M:%S') == '30:00']

### Importing simulated irradiance from bifacial_radiance

In [ ]:
irr_bf1 = pd.read_csv(r'path...\filename.csv')    # Importing the irradiance results for reference cell 1
irr_bf2 = pd.read_csv(r'path...\filename.csv')    # Importing the irradiance results for reference cell 2
irr_bf3 = pd.read_csv(r'path...\filename.csv')    # Importing the irradiance results for reference cell 3

# Setting the timestamp to be the index and converting the index to date-time
irr_bf1 = irr_bf1.set_index('Unnamed: 0')         
irr_bf1.index = pd.to_datetime(irr_bf1.index)     
irr_bf2 = irr_bf2.set_index('Unnamed: 0')
irr_bf2.index = pd.to_datetime(irr_bf2.index)
irr_bf3 = irr_bf3.set_index('Unnamed: 0')
irr_bf3.index = pd.to_datetime(irr_bf3.index)

# Renaming the columns ( E - east, W - west)
irr_bf1.rename(columns = {'east_irradiance':'E - reference cell 1','west_irradiance':'W - reference cell 1'}, inplace=True) 
irr_bf2.rename(columns = {'east_irradiance':'E - reference cell 2','west_irradiance':'W - reference cell 2'}, inplace=True)
irr_bf3.rename(columns = {'east_irradiance':'E - reference cell 3','west_irradiance':'W - reference cell 3'}, inplace=True)

# Saving the irradiance from all the three reference cells into one dataframe
irradiance = pd.DataFrame()
irradiance['E - reference cell 1'] = irr_bf1['E - reference cell 1']
irradiance['W - reference cell 1'] = irr_bf1['W - reference cell 1']
irradiance['E - reference cell 2'] = irr_bf2['E - reference cell 2']
irradiance['W - reference cell 2'] = irr_bf2['W - reference cell 2']
irradiance['E - reference cell 3'] = irr_bf3['E - reference cell 3']
irradiance['W - reference cell 3'] = irr_bf3['W - reference cell 3']

irradiance = irradiance.tz_localize('UTC')                          # Localizing the timezone
irradiance.index = (irradiance.index + pd.Timedelta(minutes=-30))   # Changing the timestamp from 11:00 PM to 10:30 PM etc.

### Total average irradiance a solar panel when the bifaciality factor is taken into account

In [ ]:
# Calculating the total average irradiance on the east and the west side
irradiance['avg_east'] = irradiance[['E - reference cell 1','E - reference cell 2','E - reference cell 3']].mean(axis = 1)
irradiance['avg_west'] = irradiance[['W - reference cell 1','W - reference cell 2','W - reference cell 3']].mean(axis = 1)
irr_ref['avg_east'] = irr_ref[['ITS6_center_east','ITS4_east_east','ITS2_west_east']].mean(axis = 1)
irr_ref['avg_west'] = irr_ref[['ITS5_center_west','ITS3_east_west','ITS1_west_west']].mean(axis = 1)

# Calculating the total average irradiance for a solar panel and taking the bifaciality factor into account
irradiance['tot_irradiation'] = irradiance['avg_west'] + (0.786 * irradiance['avg_east'])
irr_ref['tot_irradiation'] = irr_ref['avg_west'] + (0.786 * irr_ref['avg_east'])

### Importing the measured DC and AC power

In [ ]:
ac_inverter = pd.read_csv(r'path...\filename.csv')
dc_inverter = pd.read_csv(r'path...\filename.csv')

ac_inverter = ac_inverter.set_index('Time')
ac_inverter.index = pd.to_datetime(ac_inverter.index)
ac_inverter = ac_inverter.tz_localize('CET', ambiguous = False, nonexistent = 'NaT')
ac_inverter.loc[ac_inverter.index.dropna()]
ac_inverter = ac_inverter.tz_convert('UTC')

dc_inverter = dc_inverter.set_index('Time')
dc_inverter.index = pd.to_datetime(dc_inverter.index)
dc_inverter = dc_inverter.tz_localize('CET', ambiguous = False, nonexistent = 'NaT')
dc_inverter.loc[dc_inverter.index.dropna()]
dc_inverter = dc_inverter.tz_convert('UTC')

ac_inverter = ac_inverter.loc[ac_inverter.index.strftime('%M:%S') == '30:00']
dc_inverter = dc_inverter.loc[dc_inverter.index.strftime('%M:%S') == '30:00']

### Selecting timestamps for the power simulation

In [ ]:
# Definining start and end time 
start = '2023-03-01 07:30:00'
ende = '2023-03-01 16:30:00'

# Selecting data from the dataframes
temp = temp[start:end]
irr_ref = irr_ref[start:end]
irradiance = irradiance[start:end]
ac_inverter = ac_inverter[start:end]
dc_inverter = dc_inverter[start:end]

### Simulating the DC power with pvwatts

In [ ]:
temp_cell = temp['average']                        # The avarage temperature
pdc0 = 40*36                                       # The installed capacity on the test site with 36 solar panels
gamma_pdc = -0.00370                               # [1/C]

g_poa_effective_ref = irr_ref['tot_irradiation']   # The total average irradiance from measured irradiance
g_poa_effective = irradiance['tot_irradiation']    # The total average irradiance from simulated irradiance

# DC power from measured irradiance
dc_power_pvwatts_ref = pvlib.pvsystem.pvwatts_dc(g_poa_effective, temp_cell, pdc0, gamma_pdc)

# DC power from simulated irradiance
dc_power_pvwatts = pvlib.pvsystem.pvwatts_dc(g_poa_effective, temp_cell, pdc0, gamma_pdc)

### Simulating the DC power with singlediode

In [ ]:
celltype = 'monoSi'
v_mp = 7.981                 # Voltage [V] at MPP 
i_mp = 4.944                 # Current [A] at MPP
v_oc = 9.538                 # Open cicuit voltage [V]
i_sc = 5.187                 # Short circuit current [A]
alpha_sc = 0.0006 * 5.187    # Temperature coefficient at short circuit [A/C]
beta_voc = -0.003 * 9.538    # Temperature coefficient at open circuit [V/C]
gamma_pmp = -0.370           # Temperature coefficient at MPP [%/C]
cells_in_series = 14

measurements = pvlib.ivtools.sdm.fit_cec_sam(celltype, v_mp, i_mp, v_oc, i_sc, alpha_sc, beta_voc, gamma_pmp, cells_in_series)

I_L_ref = measurements[0]                               # Light generated current [A] at 25 degrees
I_o_ref = measurements[1]                               # Diode reverse saturation current [A] at 25 degrees
R_s = measurements[2]                                   # Series resistance [ohm] at 25 degrees
R_sh_ref = measurements[3]                              # Shunt resistance [ohm] at 25 degrees 
a_ref = measurements[4]                                 # Adjustment of temperature coefficient
adjust = measurements[5]                                

In [ ]:
# Simulating the DC power from measured irradiance

effective_irradiance_ref = irr_ref['tot_irradiation'] 
temp_cell = temp['average']

parameters = pvlib.pvsystem.calcparams_cec(effective_irradiance = effective_irradiance_ref, temp_cell = temp_cell, 
                                           alpha_sc = alpha_sc, a_ref = a_ref, I_L_ref = I_L_ref, I_o_ref = I_o_ref, 
                                           R_sh_ref = R_sh_ref, R_s = R_s, Adjust = adjust)

photocurrent = parameters[0]           # Light generated current [A]
saturation_current = parameters[1]     # Diode saturation current [A]
resistance_series = parameters[2]      # Series resistance [ohm]
resistance_shunt = parameters[3]       # Shunt resistance [ohm]
nNsVth = parameters[4]                 # Product of diode factor, number of cells in series, and cell thermal voltage

# Obtaining the IV curve for one single solar panel, including the DC power
dc_power_singlediode_ref = pvlib.pvsystem.singlediode(photocurrent, saturation_current, resistance_series, 
                                                      resistance_shunt, nNsVth)

# Obtaining the parameters in the IV curve for all 36 solar panels on the test site, including the DC power
dc_power_singlediode_ref36 = pd.DataFrame()
dc_power_singlediode_ref36['i_sc'] = dc_power_singlediode_ref['i_sc']
dc_power_singlediode_ref36['i_mp'] = dc_power_singlediode_ref['i_mp']
dc_power_singlediode_ref36['i_x'] = dc_power_singlediode_ref['i_x']
dc_power_singlediode_ref36['i_xx'] = dc_power_singlediode_ref['i_xx']
dc_power_singlediode_ref36['v_oc'] = 36 * dc_power_singlediode_ref['v_oc']
dc_power_singlediode_ref36['v_mp'] = 36 * dc_power_singlediode_ref['v_mp']
dc_power_singlediode_ref36['p_mp'] = 36 * dc_power_singlediode_ref['p_mp']

In [ ]:
# Simulating the DC power from simulated irradiance

effective_irradiance = irradiance['tot_irradiation'] 
temp_cell = temp['average']

parameters = pvlib.pvsystem.calcparams_cec(effective_irradiance = effective_irradiance, temp_cell = temp_cell, 
                                           alpha_sc = alpha_sc, a_ref = a_ref, I_L_ref = I_L_ref, I_o_ref = I_o_ref, 
                                           R_sh_ref = R_sh_ref, R_s = R_s, Adjust = adjust)

photocurrent = parameters[0]           
saturation_current = parameters[1]     
resistance_series = parameters[2]      
resistance_shunt = parameters[3]       
nNsVth = parameters[4]                 

dc_power_singlediode = pvlib.pvsystem.singlediode(photocurrent, saturation_current, resistance_series, 
                                                  resistance_shunt, nNsVth)

dc_power_singlediode36 = pd.DataFrame()
dc_power_singlediode36['i_sc'] = dc_power_singlediode['i_sc']
dc_power_singlediode36['i_mp'] = dc_power_singlediode['i_mp']
dc_power_singlediode36['i_x'] = dc_power_singlediode['i_x']
dc_power_singlediode36['i_xx'] = dc_power_singlediode['i_xx']
dc_power_singlediode36['v_oc'] = 36 * dc_power_singlediode['v_oc']
dc_power_singlediode36['v_mp'] = 36 * dc_power_singlediode['v_mp']
dc_power_singlediode36['p_mp'] = 36 * dc_power_singlediode['p_mp']

### Simulating the AC power for the test site when using DC power calculated with singlediode

In [ ]:
pdc0_inv = 3400                  # The inverter DC input limit
eta_inv_nom = 0.96               # Nominal efficiency of the inverter, default value
eta_inv_ref = 0.9637             # Reference efficiency of the invcerter, default value

pdc_inv_ref = dc_power_singlediode_ref36['p_mp']    # DC power into the inverter from measured irradiance
pdc_inv = dc_power_singlediode36['p_mp']            # DC power into the inverter from simulated irradiance

# Simulating the AC power from measured irradiance
ac_power_simulated_ref = pvlib.inverter.pvwatts(pdc_inv_ref, pdc0_inv, eta_inv_nom, eta_inv_ref)

# Simulating the AC power from simulated irradiance
ac_power_simulated = pvlib.inverter.pvwatts(pdc_inv, pdc0_inv, eta_inv_nom, eta_inv_ref)

### Efficiency of the inverter

In [ ]:
# Simulated inverter efficiency from simulated irradiance
simulated_efficiency_ref = (ac_power_simulated_ref/dc_power_simulated_ref['p_mp']) * 100

# Simulated inverter efficiency from measured irradiance
simulated_efficiency = (ac_power_simulated/dc_power_simulated['p_mp']) * 100

# Inverter efficiency from measured AC and DC power
efficiency_inverter = ac_inverter['Inv1 Pac (W)']/dc_inverter['Str1.0 P (W)']  *100

For the steps above, the measured irradiance from the reference cells and the simulated irradiance from bifacial_radiance have been imported to simulate the DC and AC power. The same method was used to simulate the DC and AC power for the upper and lower limit of irradiance  due to measurement uncertianty as well.

### Cumulative sky

For the results from the cumulative sky, the energy was simulated due to the monthly value for the irradiance [Wh/m2]. Therefore, the DC energy was simulated with pvwatts and the AC energy was not calculated as discussed in Section 4.8 of the master's thesis. The DC energy was simulated with the same method as the DC power.